In [1]:
from pathlib import Path

import pandas as pd
from lightning import pytorch as pl
import ray
from ray import tune
from ray.train import CheckpointConfig, RunConfig, ScalingConfig
from ray.train.lightning import (RayDDPStrategy, RayLightningEnvironment,
                                 RayTrainReportCallback, prepare_trainer)
from ray.train.torch import TorchTrainer
from ray.tune.search.hyperopt import HyperOptSearch
from ray.tune.search.optuna import OptunaSearch
from ray.tune.schedulers import FIFOScheduler

from chemprop import data, featurizers, models, nn

/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-12 00:05:44,709	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-12 00:05:45,826	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-12 00:05:47,177	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
chemprop_dir = Path.cwd().parent
train_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_zinc/train_data_jak2.csv"
test_path = "/home/aih/serra.korkmaz/projects/saturn/SurogateModel/training_data_4sm_zinc/test_data_jak2.csv"
num_workers = 0 # number of workers for dataloader. 0 means using main process for data loading
smiles_column = 'smiles' # name of the column containing SMILES strings
target_columns = ['value'] # list of names of the columns containing targets

hpopt_save_dir = Path.cwd() / "hpopt_jak2_zinc" # directory to save hyperopt results
hpopt_save_dir.mkdir(exist_ok=True)

In [3]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [4]:
# Extract SMILES and target values
train_smis = df_train[smiles_column].values
train_ys = df_train[target_columns].values

test_smis = df_test[smiles_column].values
test_ys = df_test[target_columns].values

# Convert data to MoleculeDatapoint format
train_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(train_smis, train_ys)]
test_data = [data.MoleculeDatapoint.from_smi(smi, y) for smi, y in zip(test_smis, test_ys)]


In [5]:
# Initialize featurizer
featurizer = featurizers.SimpleMoleculeMolGraphFeaturizer()

# Create training dataset
train_dset = data.MoleculeDataset(train_data, featurizer)
scaler = train_dset.normalize_targets()

# Split validation set from training data (80% train, 20% validation)
split_idx = int(len(train_data) * 0.8)
val_data = train_data[split_idx:]
train_data = train_data[:split_idx]

# Create validation and test datasets
val_dset = data.MoleculeDataset(val_data, featurizer)
val_dset.normalize_targets(scaler)

test_dset = data.MoleculeDataset(test_data, featurizer)

In [6]:
def train_model(config, train_dset, val_dset, num_workers, scaler):

    # config is a dictionary containing hyperparameters used for the trial
    depth = int(config["depth"])
    ffn_hidden_dim = int(config["ffn_hidden_dim"])
    ffn_num_layers = int(config["ffn_num_layers"])
    message_hidden_dim = int(config["message_hidden_dim"])

    train_loader = data.build_dataloader(train_dset, num_workers=num_workers, shuffle=True)
    val_loader = data.build_dataloader(val_dset, num_workers=num_workers, shuffle=False)

    mp = nn.BondMessagePassing(d_h=message_hidden_dim, depth=depth)
    agg = nn.MeanAggregation()
    output_transform = nn.UnscaleTransform.from_standard_scaler(scaler)
    ffn = nn.RegressionFFN(output_transform=output_transform, input_dim=message_hidden_dim, hidden_dim=ffn_hidden_dim, n_layers=ffn_num_layers)
    batch_norm = True
    metric_list = [nn.metrics.RMSE(), nn.metrics.MAE()]
    model = models.MPNN(mp, agg, ffn, batch_norm, metric_list)

    trainer = pl.Trainer(
        accelerator="auto",
        devices=1,
        max_epochs=20, # number of epochs to train for
        # below are needed for Ray and Lightning integration
        strategy=RayDDPStrategy(),
        callbacks=[RayTrainReportCallback()],
        plugins=[RayLightningEnvironment()],
    )

    trainer = prepare_trainer(trainer)
    trainer.fit(model, train_loader, val_loader)

In [7]:
search_space = {
    "depth": tune.qrandint(lower=2, upper=6, q=1),
    "ffn_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
    "ffn_num_layers": tune.qrandint(lower=1, upper=3, q=1),
    "message_hidden_dim": tune.qrandint(lower=300, upper=2400, q=100),
}

In [8]:
ray.init()

scheduler = FIFOScheduler()

# Scaling config controls the resources used by Ray
scaling_config = ScalingConfig(
    num_workers=1,
    use_gpu=False, # change to True if you want to use GPU
)

# Checkpoint config controls the checkpointing behavior of Ray
checkpoint_config = CheckpointConfig(
    num_to_keep=1, # number of checkpoints to keep
    checkpoint_score_attribute="val_loss", # Save the checkpoint based on this metric
    checkpoint_score_order="min", # Save the checkpoint with the lowest metric value
)

run_config = RunConfig(
    checkpoint_config=checkpoint_config,
    storage_path=hpopt_save_dir / "ray_results", # directory to save the results
)

ray_trainer = TorchTrainer(
    lambda config: train_model(
        config, train_dset, val_dset, num_workers, scaler
    ),
    scaling_config=scaling_config,
    run_config=run_config,
)

search_alg = HyperOptSearch(
    n_initial_points=1, # number of random evaluations before tree parzen estimators
    random_state_seed=42,
)

# OptunaSearch is another search algorithm that can be used
# search_alg = OptunaSearch()

tune_config = tune.TuneConfig(
    metric="val_loss",
    mode="min",
    num_samples=2, # number of trials to run
    scheduler=scheduler,
    search_alg=search_alg,
    trial_dirname_creator=lambda trial: str(trial.trial_id), # shorten filepaths

)

tuner = tune.Tuner(
    ray_trainer,
    param_space={
        "train_loop_config": search_space,
    },
    tune_config=tune_config,
)

# Start the hyperparameter search
results = tuner.fit()

(RayTrainWorker pid=2715599) Setting up process group for: env:// [rank=0, world_size=1]
(TorchTrainer pid=2715363) Started distributed worker processes: 
(TorchTrainer pid=2715363) - (node_id=ce59ddc97509b0380c09cda2b43f5e894944e3796752f02ea2dd0f2d, ip=10.233.0.55, pid=2715599) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=2715599) /home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/ray/train/lightning/_lightning_utils.py:262: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
(RayTrainWorker pid=2715599) `get_trial_name` is deprecated because the concept of a `Trial` will soon be removed in Ray Train.Ray Train will no longer assume that it's running within a Ray Tune `Trial` in the future. See this issue for more context and migration options: https://github.com/ray-project/ray/issues/49454. Disable these warnings by setting the envi

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]


(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599)   | Name            | Type               | Params | Mode 
(RayTrainWorker pid=2715599) ---------------------------------------------------------------
(RayTrainWorker pid=2715599) 0 | message_passing | BondMessagePassing | 579 K  | train
(RayTrainWorker pid=2715599) 1 | agg             | MeanAggregation    | 0      | train
(RayTrainWorker pid=2715599) 2 | bn              | BatchNorm1d        | 1.0 K  | train
(RayTrainWorker pid=2715599) 3 | predictor       | RegressionFFN      | 5.0 M  | train
(RayTrainWorker pid=2715599) 4 | X_d_transform   | Identity           | 0      | train
(RayTrainWorker pid=2715599) 5 | metrics         | ModuleList         | 0      | train
(RayTrainWorker pid=2715599) ---------------------------------------------------------------
(RayTrainWorker pid=2715599) 5.6 M     Trainable params
(RayTrainWorker pid=2715599) 0         Non-trainable params
(RayTrainWorker pid=2715599) 5.6 M     Total params
(RayTra

Epoch 0:  61%|██████    | 17/28 [00:05<00:03,  3.19it/s, v_num=3.41e+7, train_loss_step=0.485]


(TorchTrainer pid=2715600) Started distributed worker processes: 
(TorchTrainer pid=2715600) - (node_id=ce59ddc97509b0380c09cda2b43f5e894944e3796752f02ea2dd0f2d, ip=10.233.0.55, pid=2715837) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=2715837) Setting up process group for: env:// [rank=0, world_size=1]


...

(RayTrainWorker pid=2715837) `Trainer.fit` stopped: `max_epochs=20` reached.
(TorchTrainer pid=2715600) *** SIGSEGV received at time=1741734627 on cpu 21 ***
(TorchTrainer pid=2715600) PC: @     0x7f52359788fe  (unknown)  ray::gcs::TaskInfoAccessor::AsyncAddTaskEventData()
(TorchTrainer pid=2715600)     @     0x7f5236c3e6f0       3680  (unknown)
(TorchTrainer pid=2715600)     @     0x7f5235859ab5       1392  ray::core::worker::TaskEventBufferImpl::FlushEvents()
(TorchTrainer pid=2715600)     @     0x7f52357e282c       1488  ray::core::CoreWorker::Disconnect()
(TorchTrainer pid=2715600)     @     0x7f52357e2bdd       1152  ray::core::CoreWorker::ForceExit()
(TorchTrainer pid=2715600)     @     0x7f52357e300f       1680  ray::core::CoreWorker::HandleKillActor()
(TorchTrainer pid=2715600)     @     0x7f52357da514        192  ray::rpc::ServerCallImpl<>::HandleRequestImpl()
(TorchTrainer pid=2715600)     @     0x7f5235b67d08       1168  EventTracker::RecordExecution()
(TorchTrainer pid=2715

Epoch 18: 100%|██████████| 28/28 [00:09<00:00,  2.97it/s, v_num=3.41e+7, train_loss_step=0.199, val_loss=0.220, train_loss_epoch=0.183] [repeated 4x across cluster]
Validation: |          | 0/? [00:00<?, ?it/s]
(RayTrainWorker pid=2715599) 
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
Validation DataLoader 0:  33%|███▎      | 2/6 [00:00<00:00,  5.62it/s] [repeated 2x across cluster]
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 


2025-03-12 00:10:31,108	WARNING experiment_state.py:206 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds and may become a bottleneck. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this warning by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0). Set it to 0 to completely suppress this warning.
(RayTrainWorker pid=2715599) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_jak2_zinc/ray_results/TorchTrainer_2025-03-12_00-06-30/2594cc27/checkpoint_000018)


Epoch 19: 100%|██████████| 28/28 [00:07<00:00,  3.76it/s, v_num=3.41e+7, train_loss_step=0.151, val_loss=0.184, train_loss_epoch=0.171]
Validation: |          | 0/? [00:00<?, ?it/s]
(RayTrainWorker pid=2715599) 
Validation DataLoader 0:   0%|          | 0/6 [00:00<?, ?it/s]
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
(RayTrainWorker pid=2715599) 
Epoch 19: 100%|██████████| 28/28 [00:08<00:00,  3.16it/s, v_num=3.41e+7, train_loss_step=0.151, val_loss=0.221, train_loss_epoch=0.181]


(RayTrainWorker pid=2715599) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_jak2_zinc/ray_results/TorchTrainer_2025-03-12_00-06-30/2594cc27/checkpoint_000019)
(RayTrainWorker pid=2715599) `Trainer.fit` stopped: `max_epochs=20` reached.
2025-03-12 00:10:41,197	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_jak2_zinc/ray_results/TorchTrainer_2025-03-12_00-06-30' in 0.0061s.
2025-03-12 00:10:41,203	INFO tune.py:1041 -- Total run time: 250.66 seconds (250.38 seconds for the tuning loop).


In [9]:
results

ResultGrid<[
  Result(
    metrics={'train_loss': 0.18090607225894928, 'train_loss_step': 0.15052905678749084, 'val/rmse': 0.470052570104599, 'val/mae': 0.36145204305648804, 'val_loss': 0.22094941139221191, 'train_loss_epoch': 0.18090607225894928, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_jak2_zinc/ray_results/TorchTrainer_2025-03-12_00-06-30/2594cc27',
    filesystem='local',
    checkpoint=Checkpoint(filesystem=local, path=/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_jak2_zinc/ray_results/TorchTrainer_2025-03-12_00-06-30/2594cc27/checkpoint_000019)
  ),
  Result(
    metrics={'train_loss': 0.13374562561511993, 'train_loss_step': 0.12275725603103638, 'val/rmse': 0.3660379946231842, 'val/mae': 0.27642086148262024, 'val_loss': 0.1339838057756424, 'train_loss_epoch': 0.13374562561511993, 'epoch': 19, 'step': 560},
    path='/ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_jak2_zinc/ray_results/TorchTrainer_2

In [10]:
result_df = results.get_dataframe()
result_df

,train_loss,train_loss_step,val/rmse,val/mae,val_loss,train_loss_epoch,epoch,step,timestamp,checkpoint_dir_name,...,pid,hostname,node_ip,time_since_restore,iterations_since_restore,config/train_loop_config/depth,config/train_loop_config/ffn_hidden_dim,config/train_loop_config/ffn_num_layers,config/train_loop_config/message_hidden_dim,logdir
0,0.180906,0.150529,0.470053,0.361452,0.220949,0.180906,19,560,1741734639,checkpoint_000019,...,2715363,cpusrv45.scidom.de,10.233.0.55,239.672111,20,2,2000,2,500,2594cc27
1,0.133746,0.122757,0.366038,0.276421,0.133984,0.133746,19,560,1741734626,checkpoint_000019,...,2715600,cpusrv45.scidom.de,10.233.0.55,216.208837,20,2,2200,2,400,3f1b8347


In [11]:
# best configuration
best_result = results.get_best_result()
best_config = best_result.config
best_config['train_loop_config']

{'depth': 2,
 'ffn_hidden_dim': 2200,
 'ffn_num_layers': 2,
 'message_hidden_dim': 400}

In [12]:
# best model checkpoint path
best_result = results.get_best_result()
best_checkpoint_path = Path(best_result.checkpoint.path) / "checkpoint.ckpt"
print(f"Best model checkpoint path: {best_checkpoint_path}")

Best model checkpoint path: /ictstr01/home/aih/serra.korkmaz/projects/saturn/hopt/hpopt_jak2_zinc/ray_results/TorchTrainer_2025-03-12_00-06-30/3f1b8347/checkpoint_000019/checkpoint.ckpt


In [13]:
mpnn = models.MPNN.load_from_checkpoint(best_checkpoint_path)
mpnn

MPNN(
  (message_passing): BondMessagePassing(
    (W_i): Linear(in_features=86, out_features=400, bias=False)
    (W_h): Linear(in_features=400, out_features=400, bias=False)
    (W_o): Linear(in_features=472, out_features=400, bias=True)
    (dropout): Dropout(p=0.0, inplace=False)
    (tau): ReLU()
    (V_d_transform): Identity()
    (graph_transform): Identity()
  )
  (agg): MeanAggregation()
  (bn): BatchNorm1d(400, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (predictor): RegressionFFN(
    (ffn): MLP(
      (0): Sequential(
        (0): Linear(in_features=400, out_features=2200, bias=True)
      )
      (1): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=2200, bias=True)
      )
      (2): Sequential(
        (0): ReLU()
        (1): Dropout(p=0.0, inplace=False)
        (2): Linear(in_features=2200, out_features=1, bias=True)
      )
    )
    (criterion): MSE(task_weights=[[1.0]

In [14]:
import torch
test_loader = data.build_dataloader(test_dset, shuffle=False)
with torch.inference_mode():
    trainer = pl.Trainer(
        logger=None,
        enable_progress_bar=True,
        accelerator="cpu",
        devices=1
    )
    test_preds = trainer.predict(mpnn, test_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/core/saving.py:363: Skipping 'metrics' parameter because it is not possible to safely dump to YAML.
SLURM auto-requeueing enabled. Setting signal handlers.
/home/aih/serra.korkmaz/miniconda3/envs/saturn/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Predicting DataLoader 0: 100%|██████████| 4/4 [00:00<00:00, 11.93it/s]


In [15]:
import numpy as np
test_preds = np.concatenate(test_preds, axis=0)
df_test['preds'] = test_preds
df_test

,smiles,value,preds
0,Clc1ccc(N[C@H]2CCOC2)cc1Br,6.4,6.964708
1,CC[C@H](NC(=O)OC(C)(C)C)c1nc(C)cc(C)n1,6.7,7.003738
2,C[C@@H](c1ccc(F)cc1)N(C)C(=O)NC[C@@H]1CCCN(c2n...,8.3,8.201910
3,CCn1nc(C)c(NC(=O)N[C@H](C)c2oc3ccc(F)cc3c2C)c1C,8.8,8.631846
4,CC[S@@](=O)[C@@H]1CCC[C@H](NC(=O)NC[C@@H](C)CC...,6.7,5.744868
...,...,...,...
193,COc1ccc2c(c1)[C@H](N[C@H](c1ccc(F)cc1)C(F)(F)F...,8.5,8.129163
194,C[NH2+][C@H](COC)CSc1ccccc1Br,5.8,5.862997
195,C[S@@](=O)c1c(Cl)cc(Cl)c(N)c1[N+](=O)[O-],5.9,5.670559
196,CCc1nn(C)cc1NC(=O)C(=O)N1CC[C@@H](C[NH+](C)CC)C1,6.8,7.108873
